In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import torch

from torch.utils.data import DataLoader

from druxai.utils.data_gnn import DrugResponseDataset, DataloaderSampler
from druxai.utils.dataframe_utils import split_data_by_cell_line_ids
from druxai.utils.dataframe_utils import standardize_molecular_data_inplace
from torch_geometric.data import Batch
file_path = "/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/data/preprocessed"

In [76]:
# Load Data
data = DrugResponseDataset(file_path)
train_id, val_id, test_id = split_data_by_cell_line_ids(data.targets)
standardize_molecular_data_inplace(data, train_id=train_id, val_id=val_id, test_id=test_id)

INFO     Loaded targets with shape: (556840, 9)                                                                    
INFO     Loaded molecular data with shape: (1479, 19193)                                                           


In [77]:
data[0]

(tensor([ 0.6155, -0.2172, -0.4931,  ...,  0.1874,  0.1539, -0.3254]),
 Data(x=[17, 79], edge_index=[2, 36], edge_attr=[36, 10]),
 tensor([2.3674]),
 0)

In [78]:
def custom_collate(batch):
    gene_expression_values_batch = torch.stack([sample[0] for sample in batch])
    drug_x_batch = Batch.from_data_list([sample[1] for sample in batch])
    target_batch = torch.stack([sample[2] for sample in batch])
    idx_batch = torch.tensor([sample[3] for sample in batch])  # Convert idx to tensor

    return {
        "gene_expression_values": gene_expression_values_batch,
        "smile_graph": drug_x_batch,
        "target": target_batch,
        "idx": idx_batch
    }

In [91]:
# Dataloader Sampler
train_sampler = DataloaderSampler(train_id)

# Dataloader
train_loader = DataLoader(data, sampler=train_sampler, batch_size=2, shuffle=False, collate_fn=custom_collate)

In [92]:
for X in train_loader:
    encod = X
    break

In [93]:
encod["smile_graph"]

DataBatch(x=[55, 79], edge_index=[2, 120], edge_attr=[120, 10], batch=[55], ptr=[3])

In [94]:
encod["smile_graph"].batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1])